# DEVELOPING THE MAZE GENERATOR

Importing the libraries:

In [1]:
import numpy as np

Defining auxiliary classes:

In [2]:
class UnionFind:
    def __init__(self, n):
        self.n = n  # number of supernodes
        self.v = [i for i in range(n)]  # represents the parents (initially each disjoint set has a single vertex)

    def find(self, u):
        while u != self.v[u]:
            self.v[u] = self.v[self.v[u]]  # compression technique
            u = self.v[u]
            
        return u
    
    def union(self, u, v):
        root_u, root_v = self.find(u), self.find(v)
        
        if root_u == root_v:
            return False  # union was not performed
        else:
            self.v[root_v] = root_u
            self.n -= 1

            return True  # union was not performed

Defining the maze generator:

 - Labyrinth is represented by an adjacency list, as the resulting graph will always be sparse

In [3]:
# Class for the maze generator
# Inspired by a randomized Kruskal algorithm
class MazeGenerator:
    def __init__(self, N=100):
        self.N = N  # number os cells (default is a 10x10 grid)
        
        self.walls = self.generate_walls()
        
    def new_maze(self):
        maze = [[] for _ in range(self.N)]  # data structure to store the maze
        
        # Copy the walls (edges) and create a set for each cell
        walls = self.walls.copy()
        forest = UnionFind(self.N)
                
        # Generate a maze (represented by an adjacency list)
        while forest.n > 1:
            v, w = walls.pop(np.random.randint(0, len(walls)))
            
            if forest.union(v, w):
                maze[v].append(w)  # adding (v,w) in the maze
                maze[w].append(v)  # adding (w,v) in the maze
                
        return maze
                
    @staticmethod
    def actions(N, v):
        maze_shape = int(N**0.5)

        actions = []
        if v % maze_shape != 0:
            actions.append((v, v-1))
        if (v+1) % maze_shape != 0:
            actions.append((v, v+1))
        if v >= maze_shape:
            actions.append((v, v-maze_shape))
        if v+maze_shape < N:
            actions.append((v, v+maze_shape))

        return actions  # possible actions in cell v

    def generate_walls(self):
        walls = []
        
        # Movement represents the existence or not of a wall
        for v in range(self.N):
            walls.extend(self.actions(self.N, v))

        return walls

Testing the generator:

In [4]:
# Using the default of 100 cells

generator = MazeGenerator()

maze = generator.new_maze()

maze

[[10],
 [2],
 [3, 1],
 [2, 4],
 [3, 14],
 [6, 15],
 [5, 7],
 [6],
 [9],
 [8, 19],
 [0, 11, 20],
 [10],
 [22],
 [14],
 [24, 13, 4],
 [5, 25],
 [17],
 [16, 27],
 [28],
 [29, 9],
 [21, 30, 10],
 [20, 31, 22],
 [32, 23, 12, 21],
 [22, 24],
 [14, 23],
 [15, 26],
 [27, 25],
 [26, 28, 17],
 [29, 27, 18],
 [19, 28, 39],
 [20],
 [21],
 [22, 33, 42],
 [32, 43],
 [35, 44],
 [36, 34, 45],
 [35, 37],
 [47, 38, 36],
 [37],
 [49, 29],
 [41, 50],
 [40],
 [32],
 [53, 33],
 [34],
 [35],
 [47, 56],
 [46, 37, 57],
 [49, 58],
 [39, 48],
 [60, 51, 40],
 [50],
 [53, 62],
 [52, 63, 43],
 [64, 55],
 [54, 56, 65],
 [55, 46],
 [47, 58],
 [48, 59, 57],
 [58],
 [50, 70, 61],
 [62, 60],
 [61, 52, 72],
 [53, 64],
 [54, 63],
 [66, 55],
 [65, 67],
 [77, 68, 66],
 [67, 69],
 [79, 68],
 [80, 60],
 [81],
 [62],
 [74],
 [75, 73, 84],
 [74],
 [86, 77],
 [67, 76],
 [79],
 [69, 78],
 [90, 70],
 [71, 91],
 [83],
 [84, 82],
 [94, 83, 74],
 [86],
 [85, 76, 87],
 [86, 97],
 [98, 89],
 [88],
 [91, 80],
 [92, 90, 81],
 [91, 93],
 

Printing the results:

In [5]:
maze_shape = int(generator.N**0.5)

for row in range(maze_shape):
    for col in range(maze_shape-1):
        print('x', end='')
        
        if row*10+col in maze[row*10+col+1]:
            print(' ', end='')
        else:
            print('|', end='')
            
    print('x')
    
    if row != 9:
        for col in range(maze_shape):
            if row*10+col in maze[(row+1)*10+col]:
                print('  ', end='')
            else:
                print('_ ', end='')
                
        print()

x|x x x x|x x x|x x
  _ _ _     _ _ _   
x x|x|x x|x|x x|x|x
  _   _     _       
x x x x x|x x x x x
      _ _ _ _ _ _   
x|x|x x|x x x x x|x
_ _         _   _   
x x|x|x|x|x|x x|x x
  _ _   _ _       _ 
x x|x x|x x x|x x x
  _         _ _ _ _ 
x x x|x x|x x x x x
  _   _ _ _ _   _   
x|x|x|x x x|x x|x x
    _ _   _   _ _ _ 
x|x|x x x|x x x|x x
    _ _   _ _     _ 
x x x x x x|x x x x
